# Práctica 1 - Clasificación de palabras

In [18]:
import pandas as pd
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
#https://scikit-learn.org/stable/modules/model_evaluation.html
import matplotlib.pyplot as plt

import numpy as np
from numpy import mean

from sklearn.svm import SVC
from sklearn.svm import LinearSVC 

## Preparación de los datos
En esta primera parte de la práctica vamos a preparar los datos para poder hacer la clasificación. Para ello haremos que cada muestra siga el siguiente formato: C1 | C2 | C3 | .. | CN | Y  (siendo C una característica y Y el target).

In [19]:
#Auxiliar function that reformats df to one that is more fitting to Classification Problems
def reformat(dataFrame):
    dataFrame['y']=dataFrame.columns[0]
    dataFrame.rename(columns={dataFrame.columns[0]: 'word'}, inplace=True)
    return dataFrame;
#RawData, it has been a little bit formatted before reading the csv, to facilitate the process.
raw=pd.read_csv("data/data.csv")
#Split df
catala, angles= raw.filter(['catala'], axis=1), raw.filter(['angles'], axis=1)
#Reformating
catala=reformat(catala)
angles=reformat(angles)

#Merging
wordsDF=pd.concat([catala,angles], axis=0)
#Shuffle the rows
wordsDF = wordsDF.sample(frac=1).reset_index(drop=True)



### Caracterísiticas
Las características que hemos pensado que pueden ser útiles para la clasificación de las palabras son:
- Cantidad de caracteres (Númerica)
- Proporción de consonantes por vocal (consonantes / vocales) (Numérica)
- Contiene patrones o normas ortográficas de una lengua de las que vamos a clasificar?
    + Doble uso de vocal consecutivamente como es el caso del inglés (Categórica)
    + Acentos en caso de catalán (Categórica)
    + Contiene combinaciones de consonantes (consonant clusters) propias del inglés? (Categórica)

Referencias
- <Consonant_Clusters :https://www.aprendeinglessila.com/2013/09/consonantes-ingles-clusters/>
- <Frecuencia_De_Letras_Usadas_En_Catalan: https://es.sttmedia.com/frecuencias-de-letras-catalan>
- <Frecuencia_De_Letras_Usadas_En_Catalan: https://www3.nd.edu/~busiforc/handouts/cryptography/letterfrequencies.html>

Para añadir las columnas que representen estas características, hemos aplicado las siguientes funciones a las palabras.

In [20]:
#ratio de consonantes y vocales
def ratio (word):
    vocals=0
    for c in word:
        if isVocal(c):
            vocals+=1
    return round(vocals/len(word), 4)

#isVocal?
def isVocal(c):
    if(c=='a' or c=='e' or c=='i' or c=='o' or c=='u'):
        return True
    return 
#gotAccent?
def gotAccent(word):
    #List containing all possible accentuated chars from Catalan
    accentuatedChars=[ord('à'),ord('è'),ord('é'),ord('í'),ord('ò'),ord('ó'),ord('ú')]
    for c in word:
        if ord(c) in accentuatedChars:
            return 1
    return 0
def doubleVocal(word):
    ocurrences=["aa","ee","ii","oo","uu"]
    for oc in ocurrences:
        if word.find(oc)!=-1:
            return 1
    return 0

def enCC(word):
    ocurrences=["sch","spl","shr","squ","thr","spr","scr","sph","th","tw","sw","sk","sm"]
    for oc in ocurrences:
        if word.find(oc)!=-1:
            return 1
    return 0

#Features Adding
wordsDF['ratio']=wordsDF['word'].apply(ratio)
wordsDF['cantidadLetras']=wordsDF['word'].apply(len)
wordsDF['gotAccent']=wordsDF['word'].apply(gotAccent)
wordsDF['doubleVocal']=wordsDF['word'].apply(doubleVocal)
wordsDF['enCC']=wordsDF['word'].apply(enCC)
#Reorganize DF
wordsDF=wordsDF[['word','ratio','cantidadLetras','gotAccent','doubleVocal','enCC','y']]
wordsDF.to_csv('data/definitiveData.csv', index=False)
print(wordsDF.to_string)

<bound method DataFrame.to_string of           word   ratio  cantidadLetras  gotAccent  doubleVocal  enCC       y
0       escola  0.5000               6          0            0     0  catala
1     similars  0.3750               8          0            0     0  catala
2          nor  0.3333               3          0            0     0  angles
3        tocar  0.4000               5          0            0     0  catala
4        their  0.4000               5          0            0     1  angles
...        ...     ...             ...        ...          ...   ...     ...
1971    excite  0.5000               6          0            0     0  angles
1972     throw  0.2000               5          0            0     1  angles
1973     dress  0.2000               5          0            0     0  angles
1974     equip  0.6000               5          0            0     0  catala
1975   qüestió  0.2857               7          1            0     0  catala

[1976 rows x 7 columns]>


### Separación del conjunto de datos en entrenamiento y test
Vamos a escoger dos tercios para el conjunto de entrenamiento y el resto para el test. 

In [21]:
#First we must separate dataframe into X and y format
X=wordsDF.iloc[:,1:3]
y=wordsDF.iloc[:,6]

#Then we separate the data frame in training and test (will be used in chosen model)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)




## Modelos
En este problema creemos que el mejor modelo para clasificar las palabras va a ser una SVC lineal, ya que solamente debemos de clasificar entre dos clases. El kernel a usar podría ser uno lineal, aunque el RBF podría ser buena opción ya que tenemos un número de características bastante bajo en comparación al número de muestras. 

Los híperparámetros que deberemos de configurar en el modelo del SVM Lineal son:
- _penalty_
- _loss_
- _C_
- _max_iter_

El resto de híperparámetros no son tan necesarios ya que no se ajustan al problema. Por ejemplo _dual_  no nos va a dar ninguna diferencia, ya que es un booleano que indica que algoritmo usar. Se usa **dual = False** cuando el número de muestras es mayor al de características, que es nuestro caso. <br>Tampoco usaremos *multi_class* ya que en este experimento solo usaremos las clases "catalan" y "angles".

Para el ajuste de estos híperparámetros haremos un _nested cross-validation_ que consiste en combinar **GridSearch** junto a **K-Folding**, pero en este caso usaremos la variación _Stratified_ para mantener una distribución balanceada de las clases en cada fold. 

In [22]:
#Modelo
SVM_Lineal = LinearSVC(loss='squared_hinge', dual=False, C=50)

### K-Folding
Lo primero que debemos de hacer es decidir qué valor de _k_ vamos a utilizar. Normalmente se utilizan valores entre 5 y 10. Vamos a probar entre estos rangos y ver qué valor nos da mejores resultados.

In [24]:
def evaluate_model(cv):
    #https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation
    score = cross_val_score(SVM_Lineal,X,y,scoring='accuracy',cv=cv,n_jobs=-1)
    return mean(score),score.min(),score.max()


#Range to be tested
folds = range (5,11)
for k in folds:
    #verificar si hay que tocar shuffle y dejarlo a false
    cv=StratifiedKFold(n_splits=k, shuffle=True)
    k_mean, k_min, k_max = evaluate_model(cv)
    print(f'-> folds={k}, accuracy = {round(k_mean,4)}, ({round(k_min,4)}, {round(k_max,4)})')




-> folds=5, accuracy = 0.6083, (0.5924, 0.6228)
-> folds=6, accuracy = 0.6073, (0.5684, 0.6413)
-> folds=7, accuracy = 0.6063, (0.5532, 0.6431)
-> folds=8, accuracy = 0.6073, (0.587, 0.6235)
-> folds=9, accuracy = 0.6073, (0.5434, 0.653)
-> folds=10, accuracy = 0.6048, (0.5482, 0.7157)
